# Grouping

1. Simple grouping
2. More complex grouping
    - Grouping on more than one categorical column
    - Grouping on more than one numeric column
    - Using more than one aggregation method
3. Pivot tables (2D grouping -- on two categorical columns)
4. Stacking and unstacking -- moving things from rows to columns and back